In [1]:
import pandas as pd


In [2]:
df= pd.read_csv('fakenews_train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df= df.dropna()

In [4]:
# getting independent features
x= df.drop('label',axis=1)

In [5]:
# dependent feature
y=df['label']

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [7]:
# vocabularry size
voc_size=5000

# one hot representation

In [8]:
messages= x.copy()

In [9]:
messages.reset_index(inplace=True)

In [10]:
import nltk
import re
from nltk.corpus import stopwords

In [11]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [12]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [14]:
from tensorflow.keras.preprocessing.text import one_hot

In [15]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[2692, 3389, 693, 3449, 2500, 3206, 4006, 2586, 3322, 1622], [3649, 4813, 3039, 3605, 635, 2638, 4237], [4666, 1126, 131, 301], [2105, 148, 1548, 2541, 3353, 4416], [621, 635, 4491, 3158, 2140, 4753, 635, 654, 3455, 773], [3676, 2762, 2443, 3381, 3574, 4066, 4507, 3140, 3014, 1093, 2744, 4747, 4391, 2905, 4237], [82, 4399, 250, 2635, 3798, 1882, 1846, 495, 2223, 2872, 337], [1118, 3478, 2246, 1230, 1566, 2702, 4066, 3041, 2223, 2872, 337], [869, 143, 3798, 2944, 3894, 554, 3564, 974, 4066, 2157], [1902, 3205, 1943, 689, 3784, 1493, 4942, 4861], [1387, 4069, 1949, 1882, 513, 3932, 3234, 2154, 1824, 403, 3718], [2541, 4681, 2500, 554, 4066, 1566], [1719, 1163, 4817, 1118, 66, 3544, 2977, 4310, 1141], [1478, 1941, 2386, 4296, 2355, 4617, 1819, 2223, 2872, 337], [2813, 3503, 13, 659, 2448, 2223, 2872, 337], [1165, 2298, 4251, 4677, 213, 1436, 1516, 4228, 149, 3496], [71, 1565, 4813], [3934, 2414, 1685, 88, 4066, 45, 944, 4237], [4697, 3006, 3039, 4262, 4696, 4120, 4870, 2541, 4753], [1650

In [16]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2586 3322 1622]
 [   0    0    0 ...  635 2638 4237]
 [   0    0    0 ... 1126  131  301]
 ...
 [   0    0    0 ... 2223 2872  337]
 [   0    0    0 ... 4721 1116 2298]
 [   0    0    0 ... 4127  174 4249]]


In [18]:
## creating model
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
## creating model ( bi directional lstm model)
from tensorflow.keras.layers import Bidirectional


In [23]:
## creating model
embedding_vector_features= 40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model1.add(Bidirectional(LSTM(100)))  # one LSTM Layer with 100 neurons
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [26]:
model1.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
12250/12250 [==============================] - 21s 2ms/sample - loss: 0.2952 - acc: 0.8597 - val_loss: 0.1982 - val_acc: 0.9226
Epoch 2/10
12250/12250 [==============================] - 17s 1ms/sample - loss: 0.1391 - acc: 0.9479 - val_loss: 0.2042 - val_acc: 0.9206
Epoch 3/10
12250/12250 [==============================] - 17s 1ms/sample - loss: 0.0944 - acc: 0.9674 - val_loss: 0.2232 - val_acc: 0.9201
Epoch 4/10
12250/12250 [==============================] - 18s 2ms/sample - loss: 0.0678 - acc: 0.9762 - val_loss: 0.3053 - val_acc: 0.9147oss: 0.0689 - acc:  - ETA: 0s - loss: 0.0685 -
Epoch 5/10
12250/12250 [==============================] - 19s 2ms/sample - loss: 0.0451 - acc: 0.9834 - val_loss: 0.3630 - val_acc: 0.9163
Epoch 6/10
12250/12250 [==============================] - 19s 2ms/sample - loss: 0.0276 - acc: 0.9903 - val_loss: 0.3933 - val_acc: 0.9102
Epoch 7/10
12250/12250 

In [29]:
y_pred1= model1.predict_classes(x_test)

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(y_test,y_pred1)
print(cm)

[[3124  265]
 [ 277 2369]]


In [31]:
ac= accuracy_score(y_test,y_pred1)
print(ac)

0.9101905550952776


# creating model by adding dropout layer 

In [32]:
from tensorflow.keras.layers import Dropout

In [33]:
embedding_vector_features= 40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model2.add(Bidirectional(LSTM(100)))  # one LSTM Layer with 100 neurons
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
import numpy as np
x_finall= np.array(embedded_docs)
y_finall= np.array(y)

In [35]:

x_train,x_test,y_train,y_test=train_test_split(x_finall,y_finall,test_size=0.33,random_state=0)

In [36]:
model2.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 19s 2ms/sample - loss: 0.3132 - acc: 0.8479 - val_loss: 0.1917 - val_acc: 0.9233
Epoch 2/10
12250/12250 [==============================] - 16s 1ms/sample - loss: 0.1408 - acc: 0.9464 - val_loss: 0.1995 - val_acc: 0.9205
Epoch 3/10
12250/12250 [==============================] - 18s 1ms/sample - loss: 0.1045 - acc: 0.9641 - val_loss: 0.2600 - val_acc: 0.9218
Epoch 4/10
12250/12250 [==============================] - 19s 2ms/sample - loss: 0.0784 - acc: 0.9718 - val_loss: 0.2570 - val_acc: 0.9183
Epoch 5/10
12250/12250 [==============================] - 18s 1ms/sample - loss: 0.0577 - acc: 0.9801 - val_loss: 0.3029 - val_acc: 0.9143
Epoch 6/10
12250/12250 [==============================] - 18s 1ms/sample - loss: 0.0419 - acc: 0.9862 - val_loss: 0.3617 - val_acc: 0.9162
Epoch 7/10
12250/12250 [==============================] - 18s 1ms/sample - loss: 0.0315 - acc: 0.9897 - val_loss: 0.4

In [37]:
y_pred2= model2.predict_classes(x_test)

In [38]:
cm= confusion_matrix(y_test,y_pred2)
print(cm)

[[3111  278]
 [ 232 2414]]


In [39]:
ac= accuracy_score(y_test,y_pred2)
print(ac)

0.9154929577464789
